In [1]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import math
#from surprise import SVD
#from surprise import KNNBaseline
#from surprise.model_selection import train_test_split
#from surprise.model_selection import LeaveOneOut
#from surprise import Reader
#from surprise import Dataset
#from surprise import accuracy
#from surprise.model_selection import train_test_split
from collections import defaultdict
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

/Users/rahulsharma/anaconda2/lib/python2.7/site-packages/scipy/stats/morestats.py:12: DeprecationWarning: Importing from numpy.testing.decorators is deprecated, import from numpy.testing instead.
  from numpy.testing.decorators import setastest
/Users/rahulsharma/anaconda2/lib/python2.7/site-packages/_pytest/fixtures.py:844: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  params = attr.ib(convert=attr.converters.optional(tuple))
/Users/rahulsharma/anaconda2/lib/python2.7/site-packages/_pytest/fixtures.py:846: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  ids = attr.ib(default=None, convert=_ensure_immutable_ids)


In [2]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import missingno as msno

In [3]:
% matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

In [4]:
#Classification
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

In [5]:
#regression
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

In [6]:
#model selection
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [7]:
#evaluation metrics
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error # for regression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score  # for classification

In [8]:
train = pd.read_csv('/Users/rahulsharma/Teams/Cappilary_Hackathon/train- Capillary 4/train.csv')
test = pd.read_csv('/Users/rahulsharma/Teams/Cappilary_Hackathon/test.csv')
pro_att = pd.read_csv('/Users/rahulsharma/Teams/Cappilary_Hackathon/train- Capillary 4/product_attributes.csv')
df = train.copy()
test_df = test.copy()

In [9]:
df.head()

,UserId,productid,Quantity,OrderDate
0,18075,12322648,1,01/04/18
1,6820,12371370,1,01/04/18
2,6820,12973004,1,01/04/18
3,6820,12657560,1,01/04/18
4,6820,11659914,1,01/04/18


In [10]:
temp = df.copy()

In [11]:
temp.drop({'OrderDate', 'Quantity'},inplace=True, axis=1)

In [12]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy']=1
    return data_dummy

In [13]:
df_temp = temp.copy()

In [14]:
temp.drop_duplicates(keep=False,inplace=True) 

In [15]:
temp.drop_duplicates(subset=None, keep='first', inplace=True)

In [16]:
mydf = df_temp.groupby(['UserId','productid']).size().reset_index()
mydf.rename(columns = {0: 'frequency'}, inplace = True)

In [17]:
mydf.head()

,UserId,productid,frequency
0,0,11659624,1
1,1,12406904,1
2,2,11660064,1
3,3,12360440,3
4,3,12371354,3


In [18]:
data_dummy = create_data_dummy(mydf)

In [19]:
data_dummy.head()

,UserId,productid,frequency,purchase_dummy
0,0,11659624,1,1
1,1,12406904,1,1
2,2,11660064,1,1
3,3,12360440,3,1
4,3,12371354,3,1


In [20]:
df_matrix = pd.pivot_table(mydf, values='frequency', index='UserId', columns='productid')

In [21]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

In [22]:
d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['UserId'], value_name='scaled_purchase_freq').dropna()

In [23]:
print(data_norm.shape)
data_norm.head()

(73650, 3)


,UserId,productid,scaled_purchase_freq
960,960,11139192,0.0
1145,1145,11139192,0.0
4798,4798,11139192,0.0
6331,6331,11139192,0.0
10496,10496,11139192,0.0


In [24]:
def split_data(data):
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [26]:
train_data, test_data = split_data(mydf)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [27]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [28]:
data_dummy.head()

,UserId,productid,frequency,purchase_dummy
0,0,11659624,1,1
1,1,12406904,1,1
2,2,11660064,1,1
3,3,12360440,3,1
4,3,12371354,3,1


In [29]:
name = 'popularity'
target = 'frequency'
user_id = 'UserId'
item_id = 'productid'
users_to_recommend = list(mydf['UserId'])
n_rec = 10
n_display = 30
popularity = model(tc.SFrame(mydf), name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 74915 observations with 27778 users and 3026 items.

Data prepared in: 0.14751s

74915 observations to process; with 3026 unique items.

recommendations finished on 1000/74915 queries. users per second: 57349.3

...

recommendations finished on 71000/74915 queries. users per second: 46924.4

recommendations finished on 72000/74915 queries. users per second: 46892.8

recommendations finished on 73000/74915 queries. users per second: 46717.8

recommendations finished on 74000/74915 queries. users per second: 46684.8

+--------+-----------+---------------+------+
| UserId | productid |     score     | rank |
+--------+-----------+---------------+------+
|   0    |  12936598 |      10.0     |  1   |
|   0    |  12658082 |      8.0      |  2   |
|   0    |  12371276 |     7.125     |  3   |
|   0    |  12658250 | 6.83333333333 |  4   |
|   0    |  13039152 | 6.66666666667 |  5   |
|   0    |  11360688 |      6.5      |  6   |
|   0    |  12407912 |      6.0      |  7   |
|   0    |  13653580 | 5.71428571429 |  8   |
|   0    |  12406988 |      5.0      |  9   |
|   0    |  13653602 |      5.0      |  10  |
|   1    |  12936598 |      10.0     |  1   |
|   1    |  12658082 |      8.0      |  2   |
|   1    |  12371276 |     7.125     |  3   |
|   1    |  12658250 | 6.83333333333 |  4   |
|   1    |  13039152 | 6.66666666667 |  5   |
|   1    |  11360688 |      6.5      |  6   |
|   1    |  12407912 |      6.0      |  7   |
|   1    |  13653580 | 5.71428571429 |  8   |
|   1    |  12406988 |      5.0   

In [423]:
users_to_recommend = list(test['UserId'])
n_rec = 10 
recom = pop_norm.recommend(users=users_to_recommend, k=n_rec)

recommendations finished on 1000/2350 queries. users per second: 24059.9

recommendations finished on 2000/2350 queries. users per second: 33075.3

In [424]:
df_rec = recom.to_dataframe()
temp_rec = df_rec.copy()
temp_df_df = temp_rec.groupby('UserId')['productid'].apply(list)
temp_df_df.to_csv('/Users/rahulsharma/Teams/Cappilary_Hackathon/popu3.csv')

In [419]:
name = 'popularity'
target = 'purchase_dummy'
user_id = 'UserId'
item_id = 'productid'
users_to_recommend = list(mydf['UserId'])
n_rec = 10
n_display = 30
pop_dummy = model(tc.SFrame(data_dummy), name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns frequency;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 74915 observations with 27778 users and 3026 items.

Data prepared in: 0.186557s

...

recommendations finished on 71000/74915 queries. users per second: 46221.8

recommendations finished on 72000/74915 queries. users per second: 45898.5

recommendations finished on 73000/74915 queries. users per second: 45782.9

recommendations finished on 74000/74915 queries. users per second: 45704.2

+--------+-----------+-------+------+
| UserId | productid | score | rank |
+--------+-----------+-------+------+
|   0    |  12437468 |  1.0  |  1   |
|   0    |  11659934 |  1.0  |  2   |
|   0    |  12658228 |  1.0  |  3   |
|   0    |  12407616 |  1.0  |  4   |
|   0    |  12407522 |  1.0  |  5   |
|   0    |  12407298 |  1.0  |  6   |
|   0    |  12371354 |  1.0  |  7   |
|   0    |  12360440 |  1.0  |  8   |
|   0    |  11660064 |  1.0  |  9   |
|   0    |  12406904 |  1.0  |  10  |
|   1    |  12437468 |  1.0  |  1   |
|   1    |  11659934 |  1.0  |  2   |
|   1    |  12658228 |  1.0  |  3   |
|   1    |  12407616 |  1.0  |  4   |
|   1    |  12407522 |  1.0  |  5   |
|   1    |  12407298 |  1.0  |  6   |
|   1    |  12371354 |  1.0  |  7   |
|   1    |  12360440 |  1.0  |  8   |
|   1    |  11660064 |  1.0  |  9   |
|   1    |  11659624 |  1.0  |  10  |
|   2    |  12437468 |  1.0  |  1   |
|   2    |  11659934 |  1.0  |  2   |
|   2    |  12658228 |  1.0  |  3   |
|   2    |  

In [350]:
users_to_recommend = list(test['UserId'])
n_rec = 10 
recom = pop_dummy.recommend(users=users_to_recommend, k=n_rec)
df_rec = recom.to_dataframe()
temp_rec = df_rec.copy()
temp_df_df = temp_rec.groupby('UserId')['productid'].apply(list)
temp_df_df.to_csv('/Users/rahulsharma/Teams/Cappilary_Hackathon/popu2.csv')

recommendations finished on 1000/2350 queries. users per second: 42780.7

recommendations finished on 2000/2350 queries. users per second: 49568.8

In [422]:
name = 'popularity'
target = 'scaled_purchase_freq'
user_id = 'UserId'
item_id = 'productid'
users_to_recommend = list(mydf['UserId'])
n_rec = 10
n_display = 30
pop_norm = model(tc.SFrame(data_norm), name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 73650 observations with 27633 users and 2506 items.

Data prepared in: 0.16662s

73650 observations to process; with 2506 unique items.

recommendations finished on 1000/74915 queries. users per second: 55239.5

...

recommendations finished on 71000/74915 queries. users per second: 46789.1

recommendations finished on 72000/74915 queries. users per second: 46829.3

recommendations finished on 73000/74915 queries. users per second: 46833.5

recommendations finished on 74000/74915 queries. users per second: 46882.4

+--------+-----------+----------------+------+
| UserId | productid |     score      | rank |
+--------+-----------+----------------+------+
|   0    |  13064998 | 0.666666666667 |  1   |
|   0    |  11659572 | 0.666666666667 |  2   |
|   0    |  12360396 | 0.666666666667 |  3   |
|   0    |  11407266 |      0.5       |  4   |
|   0    |  11407122 |      0.5       |  5   |
|   0    |  11360846 |      0.5       |  6   |
|   0    |  11150120 |      0.5       |  7   |
|   0    |  11150104 |      0.5       |  8   |
|   0    |  11150078 |      0.5       |  9   |
|   0    |  11149934 |      0.5       |  10  |
|   1    |  13064998 | 0.666666666667 |  1   |
|   1    |  11659572 | 0.666666666667 |  2   |
|   1    |  12360396 | 0.666666666667 |  3   |
|   1    |  11407266 |      0.5       |  4   |
|   1    |  11407122 |      0.5       |  5   |
|   1    |  11360846 |      0.5       |  6   |
|   1    |  11150120 |      0.5       |  7   |
|   1    |  11150104 |      0.5       |  8   |
|   1    |  1

In [352]:
users_to_recommend = list(test['UserId'])
n_rec = 10 
recom = pop_norm.recommend(users=users_to_recommend, k=n_rec)
df_rec = recom.to_dataframe()
temp_rec = df_rec.copy()
temp_df_df = temp_rec.groupby('UserId')['productid'].apply(list)
temp_df_df.to_csv('/Users/rahulsharma/Teams/Cappilary_Hackathon/popu3.csv')

recommendations finished on 1000/2350 queries. users per second: 39116

recommendations finished on 2000/2350 queries. users per second: 47658.8

In [205]:
name = 'cosine'
target = 'frequency'
user_id = 'UserId'
item_id = 'productid'
users_to_recommend = list(mydf['UserId'])
n_rec = 10
n_display = 30
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 59932 observations with 24804 users and 2981 items.

Data prepared in: 0.096428s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 71000/74915 queries. users per second: 14646.4

recommendations finished on 72000/74915 queries. users per second: 11104

recommendations finished on 73000/74915 queries. users per second: 11019

recommendations finished on 74000/74915 queries. users per second: 11093.8

+--------+-----------+-----------------+------+
| UserId | productid |      score      | rank |
+--------+-----------+-----------------+------+
|   0    |  12936768 |  5.73041850328  |  1   |
|   0    |  12423466 |  5.73041850328  |  2   |
|   0    |  11659492 |  5.73041850328  |  3   |
|   0    |  12437088 |  5.73041850328  |  4   |
|   0    |  13650204 |  5.73041850328  |  5   |
|   0    |  12437194 |  5.73041850328  |  6   |
|   0    |  12657656 |  5.73041850328  |  7   |
|   0    |  12407842 |  5.73041850328  |  8   |
|   0    |  12437212 |  5.73041850328  |  9   |
|   0    |  11481096 |  5.73041850328  |  10  |
|   1    |  12406894 |  0.174002349377 |  1   |
|   1    |  12657416 |  0.16064709425  |  2   |
|   1    |  11714312 |  0.115954220295 |  3   |
|   1    |  12360428 |  0.108127832413 |  4   |
|   1    |  12658218 | 0.0948072075844 |  5   |
|   1    |  11407394 |  0.091669857502 |  6   |
|   1    |  11659244 | 0.0810254812241 |  7   |
|   1    |  12988608 | 0.0748481154442 |

In [206]:
name = 'cosine'
target = 'purchase_dummy'
user_id = 'UserId'
item_id = 'productid'
users_to_recommend = list(mydf['UserId'])
n_rec = 10
n_display = 30
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns frequency;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 59932 observations with 24775 users and 2977 items.

Data prepared in: 0.100934s

...

recommendations finished on 71000/74915 queries. users per second: 14247.8

recommendations finished on 72000/74915 queries. users per second: 10954.4

recommendations finished on 73000/74915 queries. users per second: 10845

recommendations finished on 74000/74915 queries. users per second: 10928.2

+--------+-----------+-----------------+------+
| UserId | productid |      score      | rank |
+--------+-----------+-----------------+------+
|   0    |  13653598 |  0.104541659355 |  1   |
|   0    |  13653688 |  0.104541659355 |  2   |
|   0    |  12445180 |  0.104541659355 |  3   |
|   0    |  11407254 |  0.104541659355 |  4   |
|   0    |  13039786 |  0.104541659355 |  5   |
|   0    |  11460738 | 0.0905357599258 |  6   |
|   0    |  11407268 | 0.0853579044342 |  7   |
|   0    |  14122822 | 0.0853579044342 |  8   |
|   0    |  13653608 | 0.0853579044342 |  9   |
|   0    |  12995896 | 0.0838198065758 |  10  |
|   1    |  11481056 | 0.0904431343079 |  1   |
|   1    |  13072610 | 0.0904431343079 |  2   |
|   1    |  13650192 | 0.0904431343079 |  3   |
|   1    |  12445200 | 0.0783260464668 |  4   |
|   1    |  13650208 | 0.0783260464668 |  5   |
|   1    |  12407056 | 0.0783260464668 |  6   |
|   1    |  13650258 | 0.0783260464668 |  7   |
|   1    |  13650044 | 0.0783260464668 |

In [207]:
name = 'cosine'
target = 'scaled_purchase_freq'
user_id = 'UserId'
item_id = 'productid'
users_to_recommend = list(mydf['UserId'])
n_rec = 10
n_display = 30
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 58920 observations with 24570 users and 2504 items.

Data prepared in: 0.100025s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 71000/74915 queries. users per second: 18552.9

recommendations finished on 72000/74915 queries. users per second: 13800.2

recommendations finished on 73000/74915 queries. users per second: 13660.9

recommendations finished on 74000/74915 queries. users per second: 13729.4

+--------+-----------+------------------+------+
| UserId | productid |      score       | rank |
+--------+-----------+------------------+------+
|   0    |  11714428 |       0.0        |  1   |
|   0    |  12657960 |       0.0        |  2   |
|   0    |  11659770 |       0.0        |  3   |
|   0    |  12658084 |       0.0        |  4   |
|   0    |  12658452 |       0.0        |  5   |
|   0    |  12360462 |       0.0        |  6   |
|   0    |  11659708 |       0.0        |  7   |
|   0    |  11237192 |       0.0        |  8   |
|   0    |  12657706 |       0.0        |  9   |
|   0    |  12657712 |       0.0        |  10  |
|   1    |  11714428 |       0.0        |  1   |
|   1    |  12657960 |       0.0        |  2   |
|   1    |  11659770 |       0.0        |  3   |
|   1    |  12658084 |       0.0        |  4   |
|   1    |  12658452 |       0.0        |  5   |
|   1    |  12360462 |       0.0        |  6   |
|   1    |  11659708 |       0.0        |  7   |
|   1    |  11237192

In [211]:
name = 'pearson'
target = 'frequency'
user_id = 'UserId'
item_id = 'productid'
users_to_recommend = list(mydf['UserId'])
n_rec = 10
n_display = 30
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 59932 observations with 24804 users and 2981 items.

Data prepared in: 0.09718s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 71000/74915 queries. users per second: 14527.3

recommendations finished on 72000/74915 queries. users per second: 10888.9

recommendations finished on 73000/74915 queries. users per second: 10800.5

recommendations finished on 74000/74915 queries. users per second: 10870.1

+--------+-----------+---------------+------+
| UserId | productid |     score     | rank |
+--------+-----------+---------------+------+
|   0    |  12936598 |      10.0     |  1   |
|   0    |  12658082 |      8.0      |  2   |
|   0    |  12371276 |     7.125     |  3   |
|   0    |  12658250 |      7.0      |  4   |
|   0    |  12407912 | 6.83333333333 |  5   |
|   0    |  13039152 | 6.66666666667 |  6   |
|   0    |  11360688 |      6.5      |  7   |
|   0    |  13653580 |      6.5      |  8   |
|   0    |  12658338 |      6.0      |  9   |
|   0    |  13653682 |      5.75     |  10  |
|   1    |  12936598 |      10.0     |  1   |
|   1    |  12658082 |      8.0      |  2   |
|   1    |  12371276 |     7.125     |  3   |
|   1    |  12658250 |      7.0      |  4   |
|   1    |  12407912 | 6.83333333333 |  5   |
|   1    |  13039152 | 6.66666666667 |  6   |
|   1    |  11360688 |      6.5      |  7   |
|   1    |  13653580 |      6.5      |  8   |
|   1    |  12658338 |      6.0   

In [212]:
name = 'pearson'
target = 'purchase_dummy'
user_id = 'UserId'
item_id = 'productid'
users_to_recommend = list(mydf['UserId'])
n_rec = 10
n_display = 30
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns frequency;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 59932 observations with 24775 users and 2977 items.

Data prepared in: 0.098299s

...

recommendations finished on 71000/74915 queries. users per second: 23622.3

recommendations finished on 72000/74915 queries. users per second: 23687.9

recommendations finished on 73000/74915 queries. users per second: 23795.1

recommendations finished on 74000/74915 queries. users per second: 23891.7

+--------+-----------+-------+------+
| UserId | productid | score | rank |
+--------+-----------+-------+------+
|   0    |  12657746 |  0.0  |  1   |
|   0    |  11660004 |  0.0  |  2   |
|   0    |  13653672 |  0.0  |  3   |
|   0    |  11407194 |  0.0  |  4   |
|   0    |  12360438 |  0.0  |  5   |
|   0    |  12658528 |  0.0  |  6   |
|   0    |  12657972 |  0.0  |  7   |
|   0    |  12445210 |  0.0  |  8   |
|   0    |  12407772 |  0.0  |  9   |
|   0    |  13039496 |  0.0  |  10  |
|   1    |  12657746 |  0.0  |  1   |
|   1    |  11660004 |  0.0  |  2   |
|   1    |  13653672 |  0.0  |  3   |
|   1    |  11407194 |  0.0  |  4   |
|   1    |  12360438 |  0.0  |  5   |
|   1    |  12658528 |  0.0  |  6   |
|   1    |  12657972 |  0.0  |  7   |
|   1    |  12445210 |  0.0  |  8   |
|   1    |  12407772 |  0.0  |  9   |
|   1    |  13039496 |  0.0  |  10  |
|   2    |  12657746 |  0.0  |  1   |
|   2    |  11660004 |  0.0  |  2   |
|   2    |  13653672 |  0.0  |  3   |
|   2    |  

In [213]:
name = 'pearson'
target = 'scaled_purchase_freq'
user_id = 'UserId'
item_id = 'productid'
users_to_recommend = list(mydf['UserId'])
n_rec = 10
n_display = 30
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 58920 observations with 24570 users and 2504 items.

Data prepared in: 0.099296s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 71000/74915 queries. users per second: 15297.6

recommendations finished on 72000/74915 queries. users per second: 11425.3

recommendations finished on 73000/74915 queries. users per second: 11335.4

recommendations finished on 74000/74915 queries. users per second: 11419.6

+--------+-----------+-------+------+
| UserId | productid | score | rank |
+--------+-----------+-------+------+
|   0    |  12437280 |  1.0  |  1   |
|   0    |  12936586 |  1.0  |  2   |
|   0    |  11149934 |  1.0  |  3   |
|   0    |  13650182 |  1.0  |  4   |
|   0    |  12936644 |  1.0  |  5   |
|   0    |  11659948 |  1.0  |  6   |
|   0    |  11407434 |  1.0  |  7   |
|   0    |  12360402 |  1.0  |  8   |
|   0    |  12936754 |  1.0  |  9   |
|   0    |  11660042 |  1.0  |  10  |
|   1    |  12437280 |  1.0  |  1   |
|   1    |  12936586 |  1.0  |  2   |
|   1    |  11149934 |  1.0  |  3   |
|   1    |  13650182 |  1.0  |  4   |
|   1    |  12936644 |  1.0  |  5   |
|   1    |  11659948 |  1.0  |  6   |
|   1    |  11407434 |  1.0  |  7   |
|   1    |  12360402 |  1.0  |  8   |
|   1    |  12936754 |  1.0  |  9   |
|   1    |  11660042 |  1.0  |  10  |
|   2    |  12437280 |  1.0  |  1   |
|   2    |  12936586 |  1.0  |  2   |
|   2    |  11149934 |  1.0  |  3   |
|   2    |  

In [214]:
models_w_counts = [popularity, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]
names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [215]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/10109 queries. users per second: 23143.3

recommendations finished on 2000/10109 queries. users per second: 26625.5

recommendations finished on 3000/10109 queries. users per second: 25914.3

recommendations finished on 4000/10109 queries. users per second: 26248.4

...

recommendations finished on 7000/10058 queries. users per second: 21758.8

recommendations finished on 8000/10058 queries. users per second: 22252.7

recommendations finished on 9000/10058 queries. users per second: 22586.7

recommendations finished on 10000/10058 queries. users per second: 22949.7


Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--------+-------------------+-------------------+
|   1    | 9.94233446013e-05 | 9.94233446013e-05 |
|   2    | 9.94233446013e-05 | 0.000119308013522 |
|   3    | 0.000132564459468 | 0.000243587194273 |
|   4    | 0.000149135016902 |  0.00025373202027 |
|   5    | 0.000159077351362 | 0.000303646184108 |
|   6    | 0.000165705574336 | 0.000427925364859 |
|   7    | 0.000156236684373 | 0.000527348709461 |
|   8    | 0.000149135016902 | 0.000626772054062 |
|   9    | 0.000165705574336 | 0.000825618743265 |
|   10   | 0.000149135016902 | 0.000825618743265 |
+--------+-------------------+-------------------+
[10 rows x 3 columns]


Overall RMSE: 0.20669275015

Per User RMSE (best)
+--------+-------+------+
| UserId | count | rmse |
+--------+-------+------+
| 13821  |   1   | 0.0  |
+--------+-------+------+
[1 rows x 3 columns]


Per User

In [323]:
name = 'cosine'
target = 'scaled_purchase_freq'
user_id = 'UserId'
item_id = 'productid'
users_to_recommend = list(test['UserId'])
n_rec = 10
n_display = 30

In [34]:
name = 'cosine'
target = 'scaled_purchase_freq'
user_id = 'UserId'
item_id = 'productid'
users_to_recommend = list(test['UserId'])
n_rec = 10
n_display = 30
final_model = tc.item_similarity_recommender.create(tc.SFrame(data_norm),user_id=user_id, item_id = item_id, target=target, similarity_type=name)
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
df_rec = recom.to_dataframe()
temp_rec = df_rec.copy()
temp_df_df = temp_rec.groupby('UserId')['productid'].apply(list)

Preparing data set.

Data has 73650 observations with 27633 users and 2506 items.

Data prepared in: 0.116733s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.187916s

recommendations finished on 1000/2350 queries. users per second: 26081.1

recommendations finished on 2000/2350 queries. users per second: 31643.6

In [35]:
temp_df_df.to_csv('/Users/rahulsharma/Teams/Cappilary_Hackathon/cosi3.csv')

In [327]:
df_rec

,UserId,productid,score,rank
0,2,11141328,0.000000,1
1,2,11141326,0.000000,2
2,2,11141324,0.000000,3
3,2,11141320,0.000000,4
4,2,11141306,0.000000,5
5,2,11139650,0.000000,6
6,2,11139560,0.000000,7
7,2,11139524,0.000000,8
8,2,11139194,0.000000,9
9,2,11139192,0.000000,10


In [227]:
samp = pd.read_csv('/Users/rahulsharma/Teams/Cappilary_Hackathon/sample_submi.csv')

In [320]:
test_sub

NameError: name 'test_sub' is not defined

In [238]:
temp_df_df = temp_rec.groupby('UserId')['productid'].apply(list)

In [302]:
test.head()

,UserId
0,2
1,28
2,36
3,38
4,41


In [284]:
temp_df_df.to_csv('/Users/rahulsharma/Teams/Cappilary_Hackathon/submi_1.csv')

In [309]:
test_submi1 = pd.read_csv('/Users/rahulsharma/Teams/Cappilary_Hackathon/submi_1.csv')

In [311]:
test_submi1.shape

(2350, 2)

In [312]:
test.shape

(2350, 1)

In [299]:
test_submi1.head()

,2,"[11141328, 11141326, 11141324, 11141320, 11141306, 11139650, 11139560, 11139524, 11139194, 11139192]"
0,28,"[11149964, 11148980, 11148920, 11148854, 11145..."
1,36,"[11141328, 11141326, 11141324, 11141320, 11141..."
2,38,"[11141328, 11141326, 11141324, 11141320, 11141..."
3,41,"[11141328, 11141326, 11141324, 11141320, 11141..."
4,52,"[11141328, 11141326, 11141324, 11141320, 11141..."


In [288]:
test.shape

(2350, 1)

In [260]:
temp_df_df.to_csv('/Users/rahulsharma/Teams/Cappilary_Hackathon/test_sub.csv', index=False)

In [261]:
test_su = pd.read_csv('/Users/rahulsharma/Teams/Cappilary_Hackathon/test_sub.csv')

In [267]:
test_su.rename(columns={'[11141328, 11141326, 11141324, 11141320, 11141306, 11139650, 11139560, 11139524, 11139194, 11139192]': 'product_list'}, inplace=True)

In [269]:
test_su

,product_list
0,"[11149964, 11148980, 11148920, 11148854, 11145..."
1,"[11141328, 11141326, 11141324, 11141320, 11141..."
2,"[11141328, 11141326, 11141324, 11141320, 11141..."
3,"[11141328, 11141326, 11141324, 11141320, 11141..."
4,"[11141328, 11141326, 11141324, 11141320, 11141..."
5,"[11141328, 11141326, 11141324, 11141320, 11141..."
6,"[11141328, 11141326, 11141324, 11141320, 11141..."
7,"[11141328, 11141326, 11141324, 11141320, 11141..."
8,"[11149964, 11148980, 11148920, 11148854, 11145..."
9,"[11149964, 11148980, 11148920, 11148854, 11145..."


In [271]:
test_temp['product_list'] = test_su['product_list']

In [272]:
test_temp

,UserId,product_list
0,2,"[11149964, 11148980, 11148920, 11148854, 11145..."
1,28,"[11141328, 11141326, 11141324, 11141320, 11141..."
2,36,"[11141328, 11141326, 11141324, 11141320, 11141..."
3,38,"[11141328, 11141326, 11141324, 11141320, 11141..."
4,41,"[11141328, 11141326, 11141324, 11141320, 11141..."
5,52,"[11141328, 11141326, 11141324, 11141320, 11141..."
6,67,"[11141328, 11141326, 11141324, 11141320, 11141..."
7,69,"[11141328, 11141326, 11141324, 11141320, 11141..."
8,80,"[11149964, 11148980, 11148920, 11148854, 11145..."
9,97,"[11149964, 11148980, 11148920, 11148854, 11145..."


In [275]:
test_temp.to_csv('/Users/rahulsharma/Teams/Cappilary_Hackathon/submi_2.csv', index=False)

In [278]:
samp.to_csv('/Users/rahulsharma/Teams/Cappilary_Hackathon/submi_3.csv')

In [279]:
samp_1 = pd.read_csv('/Users/rahulsharma/Teams/Cappilary_Hackathon/sample_submi.csv')

In [280]:
samp_1

,UserId,product_list
0,2,"[12658382, 12658382, 12658382, 12658382, 12658..."
1,28,"[12658382, 12658382, 12658382, 12658382, 12658..."
2,36,"[12658382, 12658382, 12658382, 12658382, 12658..."
3,38,"[12658382, 12658382, 12658382, 12658382, 12658..."
4,41,"[12658382, 12658382, 12658382, 12658382, 12658..."
5,52,"[12658382, 12658382, 12658382, 12658382, 12658..."
6,67,"[12658382, 12658382, 12658382, 12658382, 12658..."
7,69,"[12658382, 12658382, 12658382, 12658382, 12658..."
8,80,"[12658382, 12658382, 12658382, 12658382, 12658..."
9,97,"[12658382, 12658382, 12658382, 12658382, 12658..."


In [356]:
pro_att.head()

,productid,attribute_name,attributevalue
0,11145600,Fit,37
1,11145600,Sleeve Length,23
2,11145600,Fabric,16
3,11145600,Color,25
4,11145600,Neckline,51


In [357]:
mydf.head()

,UserId,productid,frequency
0,0,11659624,1
1,1,12406904,1
2,2,11660064,1
3,3,12360440,3
4,3,12371354,3


In [358]:
pro_att[pro_att['productid']==11145600]

,productid,attribute_name,attributevalue
0,11145600,Fit,37
1,11145600,Sleeve Length,23
2,11145600,Fabric,16
3,11145600,Color,25
4,11145600,Neckline,51
547,11145600,Category,1
1213,11145600,Season,42


In [360]:
mydf[mydf['productid']==12437280]

,UserId,productid,frequency
8597,3295,12437280,1
70862,26784,12437280,3


In [361]:
train[train['productid']==12437280]

,UserId,productid,Quantity,OrderDate
43186,26784,12437280,1,22/06/18
61407,26784,12437280,1,08/07/18
73070,3295,12437280,1,19/07/18
94057,26784,12437280,1,10/08/18


In [362]:
pro_att[pro_att['productid']==12437280]

,productid,attribute_name,attributevalue
10365,12437280,Category,29
10366,12437280,Season,5
10611,12437280,Fabric,2
10612,12437280,Color,38


In [364]:
pro_att['attributevalue'].value_counts()

0      1501
1      1478
2      1260
3      1176
4      1133
5       863
6       709
7       702
8       659
9       530
10      458
11      431
12      393
13      363
14      325
15      324
16      301
17      266
18      245
19      241
20      236
21      225
22      222
23      220
24      201
25      185
26      184
28      168
27      168
29      154
       ... 
175       1
191       1
201       1
185       1
169       1
232       1
211       1
227       1
180       1
196       1
212       1
228       1
181       1
197       1
213       1
229       1
182       1
198       1
214       1
230       1
167       1
207       1
199       1
215       1
231       1
168       1
184       1
200       1
216       1
183       1
Name: attributevalue, Length: 234, dtype: int64

In [365]:
from sklearn import preprocessing

In [368]:
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(pro_att['attributevalue'])

ValueError: Expected 2D array, got 1D array instead:
array=[37. 23. 16. ...  8.  8. 24.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [369]:
for index, row in pro_att.iterrows():
    val = row['attributevalue']
    

,productid,attribute_name,attributevalue
0,11145600,Fit,37
1,11145600,Sleeve Length,23
2,11145600,Fabric,16
3,11145600,Color,25
4,11145600,Neckline,51
5,11145602,Fit,37
6,11145602,Sleeve Length,3
7,11145602,Fabric,58
8,11145602,Color,17
9,11145602,Neckline,4


In [379]:
proatt_list_1 = pro_att.groupby('productid')['attribute_name'].apply(list)

In [383]:
proatt_list_1

productid
11139192               [Fit, Fabric, Color, Category, Season]
11139194               [Fit, Fabric, Color, Category, Season]
11139524    [Fit, Sleeve Length, Fabric, Color, Neckline, ...
11139560    [Fit, Sleeve Length, Fabric, Color, Neckline, ...
11139588               [Fit, Fabric, Color, Category, Season]
11139650    [Fit, Sleeve Length, Fabric, Color, Neckline, ...
11141306                    [Fabric, Color, Category, Season]
11141308               [Fit, Fabric, Color, Category, Season]
11141318               [Fit, Fabric, Color, Category, Season]
11141320               [Fit, Fabric, Color, Category, Season]
11141324               [Fit, Fabric, Color, Category, Season]
11141326               [Fit, Fabric, Category, Season, Color]
11141328                    [Fabric, Color, Category, Season]
11141330               [Fit, Fabric, Color, Category, Season]
11141338               [Fit, Fabric, Color, Category, Season]
11141340               [Fit, Fabric, Color, Category, Season

In [381]:
proatt_list_2 = pro_att.groupby('productid')['attributevalue'].apply(list)

In [382]:
proatt_list_2

productid
11139192                [0, 2, 41, 1, 42]
11139194                [0, 2, 25, 1, 42]
11139524       [14, 23, 16, 95, 4, 1, 42]
11139560       [14, 127, 2, 41, 4, 1, 42]
11139588               [0, 58, 25, 1, 42]
11139650        [14, 23, 2, 31, 4, 1, 42]
11141306                   [2, 28, 1, 56]
11141308                [0, 2, 71, 1, 56]
11141318               [0, 2, 149, 1, 56]
11141320               [0, 2, 149, 1, 56]
11141324               [0, 2, 163, 1, 56]
11141326               [0, 2, 1, 56, 126]
11141328                  [2, 103, 1, 56]
11141330                [0, 2, 38, 1, 56]
11141338                [0, 2, 61, 1, 56]
11141340               [0, 2, 115, 1, 56]
11141342                [0, 2, 32, 1, 56]
11141346                   [2, 22, 1, 56]
11141354                   [2, 17, 1, 56]
11141530       [0, 21, 16, 25, 51, 1, 42]
11141538        [0, 23, 2, 26, 1, 42, 43]
11141644                   [2, 22, 1, 56]
11144136        [0, 21, 2, 43, 49, 1, 42]
11144260       [0, 21, 1

In [384]:
dff = pd.get_dummies(data=pro_att)

In [385]:
dff

,productid,attributevalue,attribute_name_Category,attribute_name_Collection,attribute_name_Color,attribute_name_Fabric,attribute_name_Fit,attribute_name_Material,attribute_name_Neckline,attribute_name_Season,attribute_name_Sleeve Length
0,11145600,37,0,0,0,0,1,0,0,0,0
1,11145600,23,0,0,0,0,0,0,0,0,1
2,11145600,16,0,0,0,1,0,0,0,0,0
3,11145600,25,0,0,1,0,0,0,0,0,0
4,11145600,51,0,0,0,0,0,0,1,0,0
5,11145602,37,0,0,0,0,1,0,0,0,0
6,11145602,3,0,0,0,0,0,0,0,0,1
7,11145602,58,0,0,0,1,0,0,0,0,0
8,11145602,17,0,0,1,0,0,0,0,0,0
9,11145602,4,0,0,0,0,0,0,1,0,0


In [387]:
mydf['frequency'].value_counts()

1      61898
2       7435
3       2196
4        857
5        378
6        285
7        155
8        149
9        111
10       110
12        87
11        78
14        67
13        66
16        62
15        60
17        58
19        54
18        51
20        48
22        45
24        36
21        34
23        32
25        32
26        30
32        30
33        25
27        25
31        24
       ...  
74         2
75         2
91         2
96         2
88         2
84         2
86         2
106        1
128        1
62         1
272        1
182        1
80         1
115        1
112        1
85         1
132        1
143        1
77         1
101        1
70         1
142        1
199        1
97         1
81         1
95         1
156        1
87         1
92         1
127        1
Name: frequency, Length: 106, dtype: int64

In [391]:
mydf.shape

(74915, 3)

In [390]:
pro_att[pro_att['productid']==11660064]

,productid,attribute_name,attributevalue
4107,11660064,Fit,14
4108,11660064,Sleeve Length,23
4109,11660064,Fabric,55
4110,11660064,Color,84
4111,11660064,Neckline,4
5564,11660064,Season,11
5776,11660064,Category,12


In [393]:
test.shape

(2350, 1)

In [394]:
train.head()

,UserId,productid,Quantity,OrderDate
0,18075,12322648,1,01/04/18
1,6820,12371370,1,01/04/18
2,6820,12973004,1,01/04/18
3,6820,12657560,1,01/04/18
4,6820,11659914,1,01/04/18


In [395]:
pro_att.head()

,productid,attribute_name,attributevalue
0,11145600,Fit,37
1,11145600,Sleeve Length,23
2,11145600,Fabric,16
3,11145600,Color,25
4,11145600,Neckline,51


In [396]:
mydf[mydf['UserId']==28]

,UserId,productid,frequency
55,28,11659838,1
56,28,12360404,2
57,28,12407652,1
58,28,12407706,1
59,28,13039752,1
60,28,13039778,2
61,28,13039812,1


In [397]:
 train

,UserId,productid,Quantity,OrderDate
0,18075,12322648,1,01/04/18
1,6820,12371370,1,01/04/18
2,6820,12973004,1,01/04/18
3,6820,12657560,1,01/04/18
4,6820,11659914,1,01/04/18
5,18075,12322648,1,01/04/18
6,26784,12658110,1,01/04/18
7,16920,11659532,1,01/04/18
8,26784,12407108,1,01/04/18
9,26784,11660062,1,01/04/18


In [429]:
mydf

,UserId,productid,frequency
0,0,11659624,1
1,1,12406904,1
2,2,11660064,1
3,3,12360440,3
4,3,12371354,3
5,3,12407298,3
6,3,12407522,3
7,3,12407616,3
8,3,12658228,3
9,4,11659934,1


In [426]:
aaa = mydf.groupby('UserId')['productid'].apply(list)

In [427]:
aaa

UserId
0                                               [11659624]
1                                               [12406904]
2                                               [11660064]
3        [12360440, 12371354, 12407298, 12407522, 12407...
4                                     [11659934, 12437468]
5                                     [11145740, 12658512]
6                                               [13039720]
7                                               [11659742]
8                                     [12947668, 13039546]
9                                     [12658634, 13039040]
10                                              [11460910]
11                                              [12657628]
12                                              [11360912]
13                          [11658984, 11659206, 12407424]
14                [11659298, 11660048, 11714316, 12657436]
15       [12407864, 12657716, 12657722, 13038816, 13064...
16                                    [13039094, 

In [430]:
mydf[mydf['frequency']==127]

,UserId,productid,frequency
70703,26784,12407742,127


In [432]:
pro_att

,productid,attribute_name,attributevalue
0,11145600,Fit,37
1,11145600,Sleeve Length,23
2,11145600,Fabric,16
3,11145600,Color,25
4,11145600,Neckline,51
5,11145602,Fit,37
6,11145602,Sleeve Length,3
7,11145602,Fabric,58
8,11145602,Color,17
9,11145602,Neckline,4


In [433]:
pro_att['attribute_name'].unique()

array(['Fit', 'Sleeve Length', 'Fabric', 'Color', 'Neckline', 'Category',
       'Season', 'Material', 'Collection'], dtype=object)

In [449]:
aba = pro_att.groupby('productid')['attribute_name'].apply(list)

In [446]:
aba

productid
11139192               [Fit, Fabric, Color, Category, Season]
11139194               [Fit, Fabric, Color, Category, Season]
11139524    [Fit, Sleeve Length, Fabric, Color, Neckline, ...
11139560    [Fit, Sleeve Length, Fabric, Color, Neckline, ...
11139588               [Fit, Fabric, Color, Category, Season]
11139650    [Fit, Sleeve Length, Fabric, Color, Neckline, ...
11141306                    [Fabric, Color, Category, Season]
11141308               [Fit, Fabric, Color, Category, Season]
11141318               [Fit, Fabric, Color, Category, Season]
11141320               [Fit, Fabric, Color, Category, Season]
11141324               [Fit, Fabric, Color, Category, Season]
11141326               [Fit, Fabric, Category, Season, Color]
11141328                    [Fabric, Color, Category, Season]
11141330               [Fit, Fabric, Color, Category, Season]
11141338               [Fit, Fabric, Color, Category, Season]
11141340               [Fit, Fabric, Color, Category, Season

In [447]:
aba.shape

(3015,)